# Basin Averaged AORC Precipitation 

Last update: 
August 7, 2023

Author(s): 
- Irene Garousi-Nejad: igarousi@cuahsi.org
- Tony Castronova acastronova@cuahsi.org

**Description**:
This script retrieves and calculates the basin avergare precipitation data from the AORC (Analysis of Record for Calibration) products, used for the NWM retrospective version 2.x, within a specified area and time period of interest. Note that this script is used to process a single month of hourly data. Attempting to use it for periods longer than a month could lead to time inefficiencies and potential code breakdowns.

**Data Links**: 
- Original: https://noaa-nwm-retrospective-2-1-pds.s3.amazonaws.com/index.html#forcing/
- Kerchunk: https://ciroh-nwm-zarr-retrospective-data-copy.s3.amazonaws.com/index.html#noaa-nwm-retrospective-2-1-zarr-pds/
- Description: https://hydrology.nws.noaa.gov/aorc-historic/Documents/AORC-Version1.1-SourcesMethodsandVerifications.pdf

**Software Requirements**: TODO

----

### Install and Import Packages

In [ ]:
!pip install s3fs  --quiet
!pip install kerchunk --quiet
!pip install zarr --quiet
!pip install s3fs --quiet
!pip install rioxarray --quiet
!pip install geocube --quiet

In [ ]:
import re
import dask
import numpy
import xarray
import pyproj
import pandas
import requests
import geopandas
from matplotlib import colors
import matplotlib.pyplot as plt
from dask.distributed import Client, LocalCluster
from dask.distributed import progress
import zarr
import fsspec
from pyproj import Transformer
from s3fs import S3FileSystem
from kerchunk.combine import MultiZarrToZarr
import rioxarray
import geocube
import pandas as pd
from geocube.api.core import make_geocube

### Plot the geospatial model domain

In [ ]:
# read the shapefile
gdf = geopandas.read_file(f'./GISBasins/WeberRiverBasin.shp')
gdf.plot()
plt.show()

### Define parameters

Please note that this jupyter notebook works for data within 2007-2019.

In [ ]:
# select a year of interest
year = '2015'
month_s='04'
month_e='05'

### Load Forcing Data into Memory

These data are publicly available for the entire CONUS, spanning from 1980 to 2020. Kerchunk header files have been created by the Alabama Water Institute team and this is an ongoing project. 

In [ ]:
bucket = 's3://ciroh-nwm-zarr-retrospective-data-copy/noaa-nwm-retrospective-2-1-zarr-pds/forcing/'

# create an instace of the S3FileSystem class from s3fs
s3 = S3FileSystem(anon=True)
files = s3.ls(f'{bucket}{year}')  

new_files = []
for f in files:
    parts = f.split('/')
    parts[0] += '.s3.amazonaws.com'
    parts.insert(0, 'https:/')
    new_name = '/'.join(parts)
    new_files.append(new_name)

Considering the memory limitations, it is necessary to choose a smaller subset of the dataset. Afterwards, we can utilize the MultiZarrToZarr function from the kerchunk library to merge the individual header files and generate a single kerchunk file.

In [ ]:
%%time

# select a smaller chunck of kerchunk files 
if month_s in ['01','02','03']:
    json_list = new_files[0*2190:2190] 
elif month_s in ['04','05','06']:
    json_list = new_files[2190-(4*24):2190*2]
elif month_s in ['07','08','09']:
    json_list = new_files[2190*2-(5*24):2190*3]
elif month_s in ['10','11','12']:
    json_list = new_files[2190*3-(5*24):]

mzz = MultiZarrToZarr(json_list,
    remote_protocol='s3',
    remote_options={'anon':True},
    concat_dims=['valid_time'])

d = mzz.translate()

backend_args = {"consolidated": False, "storage_options": {"fo": d}, "consolidated": False}

ds = xarray.open_dataset("reference://", engine="zarr", backend_kwargs=backend_args)

In [ ]:
ds

In [ ]:
# remove the dimension with the size of 1
ds = ds.squeeze(dim='Time')

### Add spatial metadata to the dataset

Load the National Water Model metadata dataset using xarray and add spatial metadata to it.

In [ ]:
ds_meta = xarray.open_dataset('http://thredds.hydroshare.org/thredds/dodsC/hydroshare/resources/2a8a3566e1c84b8eb3871f30841a3855/data/contents/WRF_Hydro_NWM_geospatial_data_template_land_GIS.nc')

leny = len(ds_meta.y)
x = ds_meta.x.values
y = ds_meta.y.values

ds = ds.rename({'valid_time': 'time', 'south_north':'y', 'west_east':'x'})

X, Y = numpy.meshgrid(x, y)

# define the input crs
wrf_proj = pyproj.Proj(proj='lcc',
                       lat_1=30.,
                       lat_2=60., 
                       lat_0=40.0000076293945, lon_0=-97., # Center point
                       a=6370000, b=6370000)

# define the output crs
wgs_proj = pyproj.Proj(proj='latlong', datum='WGS84')

# transform X, Y into Lat, Lon
transformer = pyproj.Transformer.from_crs(wrf_proj.crs, wgs_proj.crs)
lon, lat = transformer.transform(X, Y)

ds = ds.assign_coords(lon = (['y', 'x'], lon))
ds = ds.assign_coords(lat = (['y', 'x'], lat))
ds = ds.assign_coords(x = x)
ds = ds.assign_coords(y = y)

ds.x.attrs['axis'] = 'X'
ds.x.attrs['standard_name'] = 'projection_x_coordinate'
ds.x.attrs['long_name'] = 'x-coordinate in projected coordinate system'
ds.x.attrs['resolution'] = 1000.  # cell size

ds.y.attrs['axis'] = 'Y' 
ds.y.attrs['standard_name'] = 'projection_y_coordinate'
ds.y.attrs['long_name'] = 'y-coordinate in projected coordinate system'
ds.y.attrs['resolution'] = 1000.  # cell size

ds.lon.attrs['units'] = 'degrees_east'
ds.lon.attrs['standard_name'] = 'longitude' 
ds.lon.attrs['long_name'] = 'longitude'

ds.lat.attrs['units'] = 'degrees_north'
ds.lat.attrs['standard_name'] = 'latitude' 
ds.lat.attrs['long_name'] = 'latitude'

# add crs to netcdf file
ds.rio.write_crs(ds_meta.crs.attrs['spatial_ref'], inplace=True
                ).rio.set_spatial_dims(x_dim="x",
                                       y_dim="y",
                                       inplace=True,
                                       ).rio.write_coordinate_system(inplace=True);

In [ ]:
ds

### Add spatial reference to the model domain

In [ ]:
# convert domain geometry data into the projection of our forcing data
target_crs = pyproj.Proj(proj='lcc',
                       lat_1=30.,
                       lat_2=60., 
                       lat_0=40.0000076293945, lon_0=-97., # Center point
                       a=6370000, b=6370000) 

gdf = gdf.to_crs(target_crs.crs)

gdf['geometry'].values

Rechunk the dataset before the next steps to ensure we do not get any memory limit issue.

In [ ]:
# important step
# rechunk the dataset to solve the memory limit issue
ds = ds.chunk(chunks={'time': 1})

### Clip the CONUS-wide AORC to the extent of the model domain

Add catchment ids to the geodataset. These will be used to perform zonal statistics later on.

In [ ]:
# create zonal id column
gdf['cat'] = gdf.HUC8.astype(int)

# clip AORC to the extent of the hydrofabric geometries
ds = ds.rio.clip(gdf.geometry.values,
                 gdf.crs,
                 drop=True,
                 invert=False, from_disk=True)

# create a grid for the geocube
out_grid = make_geocube(
    vector_data=gdf,
    measurements=["cat"],
    like=ds # ensure the data are on the same grid
)

# add the catchment variable to the original dataset
ds = ds.assign_coords(cat = (['y','x'], out_grid.cat.data))

# compute the unique catchment IDs which will be used to compute zonal statistics
catchment_ids = numpy.unique(ds.cat.data[~numpy.isnan(ds.cat.data)])

print(f'The dataset contains {len(catchment_ids)} catchments')

In [ ]:
ds

### Preview the gridded catchments over the watershed vector boundary

In [ ]:
figure, ax = plt.subplots(figsize=(10,7))

# plot the gridded catchment mapping
ds.cat.plot()

# preview map geometries
gdf.iloc[:].plot(ax=ax, linewidth=2, edgecolor='k', facecolor='None')

### Run the main functions that calculate basin average precipitation data

Initiate the Dask client. This will enable us to parallelize our computations.

In [ ]:
cluster = LocalCluster(n_workers=6,
                       memory_limit='2GB')
client = Client(cluster)

First, import the `main_compute_updt.py` as a module. Then, invoke the `compute_avg_p` fucntion from this module to perform the necessary calculations. Note that the method we're using will associate grid cell with the watershed that it overlaps the most with. 

In [ ]:
%%time

import main_compute_updt

main_compute_updt.compute_avg_p(client, ds, catchment_ids, year, month_s, month_e)